
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4


In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [3]:
batch_size = 10
epochs = 10

In [4]:
linear_layer_2d = nn.Linear(in_features=128, out_features=32)
flatten = nn.Flatten()
input_2d = torch.randn(2, 2, 64)
# print(input_2d)
#print(input_2d.shape)
#output_2d = linear_layer_2d(input_2d)
#print('output_2d' , output_2d)
#print(output_2d.size())

output_1d = flatten(input_2d)
print(output_1d.size())
linear_layer_2d(output_1d)

print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)



torch.Size([2, 128])
1D  tensor([[-6.3265e-01,  4.6841e-01,  2.9163e+00,  1.1852e+00, -2.3465e-01,
         -1.3658e+00,  3.2121e+00, -3.3617e-01, -1.2112e+00, -1.0690e+00,
          7.4564e-01, -2.7932e-01,  1.6917e-02,  1.3828e+00, -2.1452e+00,
          1.3552e+00,  9.4522e-01,  6.8006e-01, -1.6999e+00,  3.5517e-01,
         -2.1268e+00, -6.8078e-01, -4.4627e-01, -1.1781e+00,  1.8261e+00,
          8.4126e-01, -1.4503e+00, -1.1020e+00,  1.4852e+00, -1.6618e+00,
         -5.2648e-01,  4.4331e-01, -7.9280e-02,  1.1845e+00,  2.2848e+00,
          9.7474e-01, -8.5390e-01,  6.7969e-01,  1.1849e+00,  4.2369e-01,
          8.6458e-01, -1.0114e+00,  3.7317e-01,  1.2369e+00, -2.1190e+00,
          9.1522e-01, -2.1001e+00, -1.0018e+00,  9.8314e-01,  4.3333e-01,
         -8.7398e-01,  2.2510e-01,  1.8060e+00, -1.0983e+00, -1.3595e+00,
         -7.8711e-01,  2.0437e+00,  9.6896e-01, -1.0954e+00, -1.3759e+00,
         -3.1704e-01, -4.8963e-01,  4.5150e-01, -1.4257e+00,  4.3882e-01,
          8.5

In [4]:

testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)


[[0.15214901 0.26409625]
 [0.15847385 0.35434336]
 [0.17404998 0.4446534 ]
 [0.19193892 0.53497918]
 [0.21678205 0.62303933]
 [0.262472   0.70661546]
 [0.32675891 0.77644054]
 [0.40500139 0.83479593]
 [0.48582416 0.85384914]
 [0.56685146 0.84283572]
 [0.6481305  0.79481721]
 [0.71564276 0.73051458]
 [0.7716696  0.65456918]
 [0.80464692 0.56690236]
 [0.82840444 0.47454697]
 [0.8521777  0.37987876]
 [0.86440259 0.28281907]
 [0.20106432 0.21354466]
 [0.24754095 0.18147987]
 [0.30772143 0.17495049]
 [0.36313466 0.189205  ]
 [0.41614068 0.21732068]
 [0.58269505 0.21382786]
 [0.64068857 0.18878019]
 [0.70321333 0.17764091]
 [0.76562795 0.18269135]
 [0.81392966 0.22233966]
 [0.50349283 0.29655437]
 [0.50303656 0.3636261 ]
 [0.50025174 0.43299491]
 [0.4951541  0.50234799]
 [0.43711337 0.5343341 ]
 [0.46480426 0.54377418]
 [0.49713651 0.5509329 ]
 [0.52726608 0.54188616]
 [0.55509857 0.53051087]
 [0.27217955 0.29960666]
 [0.30701342 0.2790273 ]
 [0.35326978 0.27934197]
 [0.39471171 0.30737899]


In [5]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")   
    break

n_total_steps = len(train_loader)
print(f'Traing dat length {n_total_steps}')
print(X)

Shape of X [N, F, C]: torch.Size([10, 68, 2])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
Traing dat length 10
tensor([[[0.1521, 0.2641],
         [0.1585, 0.3543],
         [0.1740, 0.4447],
         ...,
         [0.5266, 0.6390],
         [0.4965, 0.6434],
         [0.4618, 0.6386]],

        [[0.1600, 0.2747],
         [0.1692, 0.3643],
         [0.1853, 0.4539],
         ...,
         [0.5299, 0.6216],
         [0.5023, 0.6262],
         [0.4701, 0.6216]],

        [[0.1554, 0.2870],
         [0.1642, 0.3744],
         [0.1778, 0.4595],
         ...,
         [0.5308, 0.6296],
         [0.5001, 0.6317],
         [0.4623, 0.6268]],

        ...,

        [[0.1495, 0.2880],
         [0.1577, 0.3761],
         [0.1708, 0.4642],
         ...,
         [0.5275, 0.6309],
         [0.4965, 0.6328],
         [0.4609, 0.6299]],

        [[0.1234, 0.2676],
         [0.1298, 0.3676],
         [0.1481, 0.4675],
         ...,
         [0.5320, 0.6485],
         [0.4987, 0.6531],
    

In [13]:
#
dataiter = iter(train_loader)
landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(136, 32)
#x = flatten(landmark)
# print(landmark.shape)
# o = linear1(landmark)
# print('linear1', x.shape)

test = torch.randn(1, 68, 2)
print("Test", test)


landmark Shape  torch.Size([10, 68, 2])
tensor([[[0.1521, 0.2641],
         [0.1585, 0.3543],
         [0.1740, 0.4447],
         ...,
         [0.5266, 0.6390],
         [0.4965, 0.6434],
         [0.4618, 0.6386]],

        [[0.1600, 0.2747],
         [0.1692, 0.3643],
         [0.1853, 0.4539],
         ...,
         [0.5299, 0.6216],
         [0.5023, 0.6262],
         [0.4701, 0.6216]],

        [[0.1554, 0.2870],
         [0.1642, 0.3744],
         [0.1778, 0.4595],
         ...,
         [0.5308, 0.6296],
         [0.5001, 0.6317],
         [0.4623, 0.6268]],

        ...,

        [[0.1495, 0.2880],
         [0.1577, 0.3761],
         [0.1708, 0.4642],
         ...,
         [0.5275, 0.6309],
         [0.4965, 0.6328],
         [0.4609, 0.6299]],

        [[0.1234, 0.2676],
         [0.1298, 0.3676],
         [0.1481, 0.4675],
         ...,
         [0.5320, 0.6485],
         [0.4987, 0.6531],
         [0.4631, 0.6481]],

        [[0.1218, 0.3187],
         [0.1287, 0.4127],
  

In [13]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
            nn.ReLU(),
        )
    

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=33, bias=True)
    (3): ReLU()
  )
)


In [14]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        print(pred)
        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
# Training
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            print(pred)



In [ ]:
for t in range(epochs):
    test(test_loader, model, criterion)

print("Done !")